### Inspect Example

In [ ]:
from absbox import Generic

test01 = Generic(
    "TEST01"
    ,{"cutoff":"2021-03-01","closing":"2021-06-15","firstPay":"2021-07-26"
     ,"payFreq":["DayOfMonth",20],"poolFreq":"MonthEnd","stated":"2030-01-01"}
    ,{'assets':[["Mortgage"
        ,{"originBalance":2200,"originRate":["fix",0.045],"originTerm":30
          ,"freq":"Monthly","type":"Level","originDate":"2021-02-01"}
          ,{"currentBalance":2200
          ,"currentRate":0.08
          ,"remainTerm":20
          ,"status":"current"}]]}
    ,(("acc01",{"balance":0}),)
    ,(("A1",{"balance":1000
             ,"rate":0.07
             ,"originBalance":1000
             ,"originRate":0.07
             ,"startDate":"2020-01-03"
             ,"rateType":{"Fixed":0.08}
             ,"bondType":{"Sequential":None}})
      ,("B",{"balance":1000
             ,"rate":0.0
             ,"originBalance":1000
             ,"originRate":0.07
             ,"startDate":"2020-01-03"
             ,"rateType":{"Fixed":0.00}
             ,"bondType":{"Equity":None}
             }))
    ,(("trusteeFee",{"type":{"fixFee":30},"feeStart":"2021-06-15"}),)
    ,{"amortizing":[
         ["payFee","acc01",['trusteeFee']]
         ,["accrueAndPayInt","acc01",["A1"]]
         ,["payPrin","acc01",["A1"]]
         ,["payPrin","acc01",["B"]]
         ,["payIntResidual","acc01","B"]
        , ["inspect","cleanUpVar1",("poolBalance",)]
     ]
     ,"cleanUp":[["inspect","cleanUpVar2",("poolBalance",)]
                ,]
     }
    ,[["CollectedInterest","acc01"]
      ,["CollectedPrincipal","acc01"]
      ,["CollectedPrepayment","acc01"]
      ,["CollectedRecoveries","acc01"]]
    ,None
    ,None
    ,None
    ,{
      "AfterCollect":{
        "Mytrigger0":{"condition":["date",">","2022-01-01"]
                      ,"effects":("actions"
                                    ,["inspect","cleanUpVar3",("poolBalance",)]
                                 )
                      ,"status":False
                      ,"curable":False}
      }
    }
    ,("PreClosing","Amortizing")
    )


In [65]:
from absbox import API,mkDeal,readInspect,EnginePath
localAPI = API(EnginePath.DEV,check=False)

Connecting engine server -> http://localhost:8081

✅Connected, local lib:0.28.5, server:0.30.4

In [66]:
r = localAPI.run(test01
                 ,runAssump = [("call", ("if", ["date",">","2022-05-01"]))]
                 ,read=True)

Warning Message from server:
Bond A1 is not paid off
Bond B is not paid off
Account acc01 has cash to be distributed

In [67]:
r['result']['status']

,Date,From,To
0,2021-06-15,PreClosing,Amortizing
1,2022-05-20,Amortizing,Called
2,2022-05-20,DealEnd,Clean Up


In [68]:
readInspect(r['result'])

,cleanUpVar1,cleanUpVar3,cleanUpVar2
Date,,,
2021-07-26,2200.00,NaN,NaN
2021-08-20,2200.00,NaN,NaN
2021-09-20,2200.00,NaN,NaN
2021-10-20,2200.00,NaN,NaN
2021-11-20,2200.00,NaN,NaN
2021-12-20,2200.00,NaN,NaN
2022-01-20,2200.00,NaN,NaN
2022-01-31,NaN,2096.80,NaN
2022-02-20,2096.80,NaN,NaN
